In [1]:
from pyspark.sql.functions import window
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
sc = SparkContext()
sc.setLogLevel("ERROR")

In [2]:
spark = SparkSession \
    .builder \
    .appName("ContarPalavrasEstruturado") \
    .getOrCreate()


In [3]:
linhas = spark.readStream \
    .format("socket")\
    .option("host", "localhost")\
    .option("port", 3456)\
    .option("includeTimestamp", "true")\
    .load()

In [5]:
palavras = linhas.select(
    explode(split(linhas.value, " ")).alias("palavra"),
    linhas.timestamp
)


In [6]:
contagem = palavras.groupBy(
window(palavras.timestamp, "10 minutes", "5 minutes"),
palavras.palavra
).count().orderBy('window')

In [7]:
consulta = contagem.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

In [ ]:
consulta.awaitTermination()